USING VIET OCR TO RECOGNIZE TEXT

In [ ]:
import torch
print(torch.__version__)


In [ ]:
import numpy as np
from vietocr.tool.predictor import Predictor
from vietocr.tool.config import Cfg
from PIL import Image
import os

In [ ]:
# Khởi tạo PaddleOCR và VietOCR
# ocr = PaddleOCR(lang='vi')
#Viet OCR
cfg = Cfg.load_config_from_name('vgg_transformer')
predictor = Predictor(cfg)

In [ ]:
folder_path = r'D:\Private\Project\kaggle\OCR\PaddleOCR\data_OCR\final_data'

In [ ]:
# import os

# files = sorted(os.listdir(folder_path))

# for index, file in enumerate(files, start=1):
#     old_path = os.path.join(folder_path, file)
#     new_name = f"img_{index}.jpg" 
#     new_path = os.path.join(folder_path, new_name)

#     os.rename(old_path, new_path)

# print("Đổi tên thành công!")


In [ ]:
import os
from PIL import Image
import numpy as np

label_folder = r'D:\Private\Project\kaggle\OCR\PaddleOCR\data_OCR\labels' 

os.makedirs(label_folder, exist_ok=True)

def create_labels(folder_path):
    folder_path = os.path.abspath(folder_path)  # Chuyển thành đường dẫn tuyệt đối
    for path in os.listdir(folder_path):
        image_path = os.path.join(folder_path, path)

        try:
            image = Image.open(image_path).convert('RGB')

            # Dự đoán văn bản trong ảnh
            text = predictor.predict(image)

            # Chuyển đường dẫn ảnh sang tương đối (giữ nguyên dấu `\` trên Windows)
            relative_image_path = os.path.relpath(image_path, os.getcwd())

            # Lưu nhãn
            label_file_path = os.path.join(label_folder, f"{os.path.splitext(path)[0]}.txt")
            with open(label_file_path, 'w', encoding='utf-8') as f:
                f.write(f"{relative_image_path}\t{text}\n")

            print(f"✅ Đã lưu nhãn cho ảnh {path} vào {label_file_path}")
            print(f"- Nội dung: {text}\n")

        except Exception as e:
            print(f"❌ Lỗi khi xử lý ảnh {path}: {e}")


create_labels(folder_path)


MERGE FILE

In [ ]:
import os

def merge_txt_files(folder_path, output_file):
    with open(output_file, "w", encoding="utf-8") as f_out:
        # Duyệt tất cả các file trong thư mục
        for file_name in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file_name)

            # Chỉ xử lý file .txt
            if file_name.endswith(".txt") and os.path.isfile(file_path):
                with open(file_path, "r", encoding="utf-8") as f_in:
                    # Ghi nội dung file vào file gộp
                    f_out.write(f_in.read())

    print(f"✅ Merged all .txt files into: {output_file}")

# Ví dụ sử dụng
training_folder = r"D:\Private\Project\kaggle\OCR\PaddleOCR\data_OCR\labels"
output_file = (r"D:\Private\Project\kaggle\OCR\PaddleOCR\dataset\train_labels.txt")

merge_txt_files(training_folder, output_file)


In [ ]:
import os

# Đường dẫn đến các thư mục
train_images_dir = r"D:\Private\Project\kaggle\OCR\PaddleOCR\data_OCR\final_data"  # Thay bằng đường dẫn thực tế
labels_dir = r"D:\Private\Project\kaggle\OCR\PaddleOCR\data_OCR\labels"              # Thay bằng đường dẫn thực tế

# Đảm bảo các thư mục tồn tại
if not os.path.exists(train_images_dir):
    print(f"Thư mục {train_images_dir} không tồn tại!")
    exit()
if not os.path.exists(labels_dir):
    print(f"Thư mục {labels_dir} không tồn tại!")
    exit()

# Lấy danh sách tên file trong train_images (chỉ lấy tên cơ bản, bỏ phần mở rộng)
image_files = set()
for image_file in os.listdir(train_images_dir):
    if image_file.lower().endswith((".jpg", ".jpeg", ".png")):  # Hỗ trợ nhiều định dạng
        image_name = os.path.splitext(image_file)[0].lower()  # Chuyển về chữ thường để so sánh
        image_files.add(image_name)

# Lấy danh sách tên file trong labels (chỉ lấy tên cơ bản, bỏ phần mở rộng)
label_files = set()
for label_file in os.listdir(labels_dir):
    if label_file.lower().endswith(".txt"):
        label_name = os.path.splitext(label_file)[0].lower()  # Chuyển về chữ thường để so sánh
        label_files.add(label_name)

# Kiểm tra sự tương ứng
missing_images = label_files - image_files  # Các label không có ảnh tương ứng
extra_images = image_files - label_files    # Các ảnh không có label tương ứng

# Báo cáo kết quả
if missing_images:
    print("Các file label không có ảnh tương ứng:")
    for name in missing_images:
        print(f"- {name}.txt (không có {name}.jpg)")
else:
    print("Tất cả file label đều có ảnh tương ứng.")

if extra_images:
    print("Các file ảnh không có label tương ứng:")
    for name in extra_images:
        print(f"- {name}.jpg (không có {name}.txt)")
else:
    print("Tất cả file ảnh đều có label tương ứng.")

# Tùy chọn: In danh sách đ

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Đường dẫn thư mục và file
data_dir = "train_data/ic15_data"
os.makedirs(data_dir, exist_ok=True)

train_dir = os.path.join(data_dir, "train")
eval_dir = os.path.join(data_dir, "eval")
test_dir = os.path.join(data_dir, "test")
os.makedirs(train_dir, exist_ok=True)
os.makedirs(eval_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Đọc file train_labels.txt
label_file = "D:/Private/Project/kaggle/OCR/PaddleOCR/dataset/train_labels.txt"
with open(label_file, "r", encoding="utf-8") as f:
    lines = f.readlines()

# Tách đường dẫn ảnh và nhãn
image_paths = []
labels = []
for line in lines:
    image_path, label = line.strip().split("\t")
    image_paths.append(image_path)
    labels.append(label)

# Tách dữ liệu thành train (80%), eval (10%), test (10%)
train_paths, temp_paths, train_labels, temp_labels = train_test_split(
    image_paths, labels, test_size=0.2, random_state=42
)
eval_paths, test_paths, eval_labels, test_labels = train_test_split(
    temp_paths, temp_labels, test_size=0.5, random_state=42
)

# Hàm di chuyển ảnh và tạo file nhãn
def move_images_and_create_labels(image_paths, labels, target_dir, label_file):
    with open(label_file, "w", encoding="utf-8") as f:
        for img_path, label in zip(image_paths, labels):
            # Tên file ảnh
            img_name = os.path.basename(img_path)
            # Đường dẫn mới
            new_img_path = os.path.join(target_dir, img_name)
            # Di chuyển ảnh
            shutil.copy(img_path, new_img_path)
            # Ghi vào file nhãn
            rel_path = os.path.join(os.path.basename(target_dir), img_name)
            f.write(f"{rel_path}\t{label}\n")

# Tạo các tập dữ liệu
move_images_and_create_labels(train_paths, train_labels, train_dir, os.path.join(data_dir, "rec_gt_train.txt"))
move_images_and_create_labels(eval_paths, eval_labels, eval_dir, os.path.join(data_dir, "rec_gt_eval.txt"))
move_images_and_create_labels(test_paths, test_labels, test_dir, os.path.join(data_dir, "rec_gt_test.txt"))

print("Dữ liệu đã được tách thành công!")

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Thư mục dữ liệu
base_dir = "D:/Private/Project/kaggle/OCR/PaddleOCR"
data_dir = os.path.join(base_dir, "train_data/ic15_data")
os.makedirs(data_dir, exist_ok=True)

train_dir = os.path.join(data_dir, "train")
eval_dir = os.path.join(data_dir, "eval")
test_dir = os.path.join(data_dir, "test")

for folder in [train_dir, eval_dir, test_dir]:
    os.makedirs(folder, exist_ok=True)

# File nhãn và thư mục ảnh
label_file = os.path.join(base_dir, "dataset/train_labels.txt")
drive_dir = os.path.join(base_dir, "data_OCR/final_data")

if not os.path.exists(label_file):
    raise FileNotFoundError(f"File nhãn không tồn tại: {label_file}")

# Đọc file nhãn
image_paths = []
labels = []

with open(label_file, "r", encoding="utf-8") as f:
    lines = f.readlines()

    for line in lines:
        try:
            image_path, label = line.strip().split("\t")
            abs_image_path = os.path.abspath(os.path.join(base_dir, image_path)).replace("\\", "/")

            if os.path.exists(abs_image_path):
                image_paths.append(abs_image_path)
                labels.append(label)
            else:
                print(f"Cảnh báo: Không tìm thấy ảnh - {abs_image_path}")
        except ValueError:
            print(f"Bỏ qua dòng lỗi định dạng: {line.strip()}")

if not image_paths:
    raise ValueError("Không có ảnh hợp lệ trong danh sách!")

# Chia tập dữ liệu
train_paths, temp_paths, train_labels, temp_labels = train_test_split(
    image_paths, labels, test_size=0.2, random_state=42
)

eval_paths, test_paths, eval_labels, test_labels = train_test_split(
    temp_paths, temp_labels, test_size=0.5, random_state=42
)

# Hàm di chuyển ảnh và tạo file nhãn
def move_images_and_create_labels(image_paths, labels, target_dir, label_file):
    with open(label_file, "w", encoding="utf-8") as f:
        for img_path, label in zip(image_paths, labels):
            try:
                img_name = os.path.basename(img_path)
                new_img_path = os.path.join(target_dir, img_name)

                shutil.copy(img_path, new_img_path)
                abs_path = os.path.abspath(new_img_path).replace("\\", "/")

                f.write(f"{abs_path}\t{label}\n")
            except Exception as e:
                print(f"Lỗi khi di chuyển ảnh {img_path}: {e}")

# Di chuyển ảnh và tạo file nhãn
move_images_and_create_labels(train_paths, train_labels, train_dir, os.path.join(data_dir, "rec_gt_train.txt"))
move_images_and_create_labels(eval_paths, eval_labels, eval_dir, os.path.join(data_dir, "rec_gt_eval.txt"))
move_images_and_create_labels(test_paths, test_labels, test_dir, os.path.join(data_dir, "rec_gt_test.txt"))

# Thống kê kết quả
print("Dữ liệu đã được tách thành công!")
print(f"Số lượng ảnh train: {len(train_paths)}")
print(f"Số lượng ảnh eval: {len(eval_paths)}")
print(f"Số lượng ảnh test: {len(test_paths)}")

for folder, name in zip([train_dir, eval_dir, test_dir], ["train", "eval", "test"]):
    print(f"Thư mục {name}: {folder}, có {len(os.listdir(folder))} ảnh")

print("Thư mục làm việc hiện tại:", os.getcwd())

# Kiểm tra thử một ảnh
check_img = os.path.join(train_dir, "img_280.jpg")
check_img = os.path.abspath(check_img).replace("\\", "/")
if os.path.exists(check_img):
    print(f"Ảnh tồn tại: {check_img}")
else:
    print(f"Ảnh không tồn tại: {check_img}")

# Mã đã tối ưu và sẵn sàng chạy 🚀

In [ ]:
# import os
# import shutil
# from sklearn.model_selection import train_test_split

# # Đường dẫn thư mục gốc (tuyệt đối)
# data_dir = r"D:\Private\Project\kaggle\OCR\PaddleOCR\train_data\ic15_data"
# os.makedirs(data_dir, exist_ok=True)

# # Đường dẫn tuyệt đối cho các thư mục train, eval, test
# train_dir = os.path.join(data_dir, "train")
# eval_dir = os.path.join(data_dir, "eval")
# test_dir = os.path.join(data_dir, "test")
# os.makedirs(train_dir, exist_ok=True)
# os.makedirs(eval_dir, exist_ok=True)
# os.makedirs(test_dir, exist_ok=True)

# # Đọc file train_labels.txt
# label_file = r"D:\Private\Project\kaggle\OCR\PaddleOCR\dataset\train_labels.txt"
# with open(label_file, "r", encoding="utf-8") as f:
#     lines = f.readlines()

# # Tách đường dẫn ảnh và nhãn
# image_paths = []
# labels = []
# for line in lines:
#     image_path, label = line.strip().split("\t")
#     image_paths.append(image_path)
#     labels.append(label)

# # Tách dữ liệu thành train (80%), eval (10%), test (10%)
# train_paths, temp_paths, train_labels, temp_labels = train_test_split(
#     image_paths, labels, test_size=0.2, random_state=42
# )
# eval_paths, test_paths, eval_labels, test_labels = train_test_split(
#     temp_paths, temp_labels, test_size=0.5, random_state=42
# )

# # Hàm di chuyển ảnh và tạo file nhãn với đường dẫn tuyệt đối
# def move_images_and_create_labels(image_paths, labels, target_dir, label_file):
#     with open(label_file, "w", encoding="utf-8") as f:
#         for img_path, label in zip(image_paths, labels):
#             # Tên file ảnh
#             img_name = os.path.basename(img_path)
#             # Đường dẫn mới (tuyệt đối)
#             new_img_path = os.path.join(target_dir, img_name)
#             # Di chuyển ảnh
#             shutil.copy(img_path, new_img_path)
#             Ghi đường dẫn tuyệt đối vào file nhãn
#             abs_path = os.path.abspath(new_img_path)  # Lấy đường dẫn tuyệt đối
#             # Thay dấu \ thành / để đồng bộ trên các hệ điều hành
#             abs_path = abs_path.replace("\\", "/")
#             f.write(f"{abs_path}\t{label}\n")

# # Tạo các tập dữ liệu
# move_images_and_create_labels(
#     train_paths, train_labels, train_dir, os.path.join(data_dir, "rec_gt_train.txt")
# )
# move_images_and_create_labels(
#     eval_paths, eval_labels, eval_dir, os.path.join(data_dir, "rec_gt_eval.txt")
# )
# move_images_and_create_labels(
#     test_paths, test_labels, test_dir, os.path.join(data_dir, "rec_gt_test.txt")
# )

# print("Dữ liệu đã được tách thành công!")

In [ ]:
import os

# Đường dẫn đến thư mục chứa các file nhãn
data_dir = r"D:\Private\Project\kaggle\OCR\PaddleOCR\train_data\ic15_data"

# Danh sách các file nhãn cần kiểm tra
label_files = [
    os.path.join(data_dir, "rec_gt_train.txt"),
    os.path.join(data_dir, "rec_gt_eval.txt"),
    os.path.join(data_dir, "rec_gt_test.txt")
]

# Hàm kiểm tra đường dẫn trong file nhãn
def check_image_paths(label_file):
    if not os.path.exists(label_file):
        print(f"File nhãn {label_file} không tồn tại!")
        return

    with open(label_file, "r", encoding="utf-8") as f:
        lines = f.readlines()

    print(f"\nKiểm tra file nhãn: {label_file}")
    missing_images = 0
    for line in lines:
        try:
            # Tách đường dẫn và nhãn
            img_path, label = line.strip().split("\t")
            # Kiểm tra xem đường dẫn có tồn tại không
            if not os.path.exists(img_path):
                print(f"Ảnh không tồn tại: {img_path}")
                missing_images += 1
            else:
                print(f"Ảnh tồn tại: {img_path}")
        except Exception as e:
            print(f"Lỗi khi xử lý dòng: {line.strip()} - Lỗi: {e}")
            missing_images += 1

    if missing_images == 0:
        print(f"Tất cả ảnh trong {label_file} đều tồn tại!")
    else:
        print(f"Tổng số ảnh không tồn tại: {missing_images}")

# Kiểm tra từng file nhãn
for label_file in label_files:
    check_image_paths(label_file)

print("\nKiểm tra hoàn tất!")

In [ ]:
import os

def check_dataset(label_file, base_dir):
    with open(label_file, "r", encoding="utf-8") as f:
        lines = f.readlines()

    valid_lines = []
    for line in lines:
        try:
            img_path, label = line.strip().split("\t")
            full_path = os.path.join(base_dir, img_path)
            if os.path.exists(full_path):
                valid_lines.append(line)
            else:
                print(f"Không tìm thấy ảnh: {full_path}")
        except Exception as e:
            print(f"Lỗi ở dòng: {line.strip()} - {e}")

    # Ghi lại file nhãn đã làm sạch
    with open(label_file + ".clean", "w", encoding="utf-8") as f:
        f.writelines(valid_lines)

# Kiểm tra cả train và eval
base_dir = "./train_data/ic15_data"
check_dataset("train_data/ic15_data/rec_gt_train.txt", base_dir)
check_dataset("train_data/ic15_data/rec_gt_eval.txt", base_dir)

In [ ]:
import os

# Đường dẫn đến các thư mục
train_images_dir = r"D:\Private\Project\kaggle\OCR\PaddleOCR\dataset\train_images"  # Thay bằng đường dẫn thực tế
labels_dir = "./labels"              # Thay bằng đường dẫn thực tế

# Đảm bảo các thư mục tồn tại
if not os.path.exists(train_images_dir):
    print(f"Thư mục {train_images_dir} không tồn tại!")
    exit()
if not os.path.exists(labels_dir):
    print(f"Thư mục {labels_dir} không tồn tại!")
    exit()

# Lặp qua tất cả file trong thư mục labels
for label_file in os.listdir(labels_dir):
    if label_file.endswith(".txt"):
        label_path = os.path.join(labels_dir, label_file)
        
        # Kiểm tra file rỗng
        if os.path.getsize(label_path) == 0:
            print(f"File rỗng: {label_path}")
            
            # Lấy tên file ảnh tương ứng (thay .txt bằng .jpg)
            image_name = os.path.splitext(label_file)[0] + ".jpg"
            image_path = os.path.join(train_images_dir, image_name)
            
            # Xóa file nhãn rỗng
            os.remove(label_path)
            print(f"Đã xóa file nhãn: {label_path}")
            
            # Xóa ảnh tương ứng nếu tồn tại
            if os.path.exists(image_path):
                os.remove(image_path)
                print(f"Đã xóa ảnh: {image_path}")
            else:
                print(f"Không tìm thấy ảnh tương ứng: {image_path}")
        else:
            print(f"File {label_path} không rỗng, bỏ qua.")

print("Kiểm tra và xóa hoàn tất!")

In [ ]:
import os

# Đường dẫn đến các thư mục
# train_images_dir = train_images_dir # Thay bằng đường dẫn thực tế
# labels_dir = "labels"              # Thay bằng đường dẫn thực tế

# Đảm bảo các thư mục tồn tại
if not os.path.exists(train_images_dir):
    print(f"Thư mục {train_images_dir} không tồn tại!")
    exit()
if not os.path.exists(labels_dir):
    print(f"Thư mục {labels_dir} không tồn tại!")
    exit()

# Lấy danh sách tên file trong train_images (chỉ lấy tên cơ bản, bỏ phần mở rộng)
image_files = set()
for image_file in os.listdir(train_images_dir):
    if image_file.lower().endswith((".jpg", ".jpeg", ".png")):  # Hỗ trợ nhiều định dạng
        image_name = os.path.splitext(image_file)[0].lower()  # Chuyển về chữ thường để so sánh
        image_files.add(image_name)

# Lấy danh sách tên file trong labels (chỉ lấy tên cơ bản, bỏ phần mở rộng)
label_files = set()
for label_file in os.listdir(labels_dir):
    if label_file.lower().endswith(".txt"):
        label_name = os.path.splitext(label_file)[0].lower()  # Chuyển về chữ thường để so sánh
        label_files.add(label_name)

# Kiểm tra sự tương ứng
missing_images = label_files - image_files  # Các label không có ảnh tương ứng
extra_images = image_files - label_files    # Các ảnh không có label tương ứng

# Báo cáo kết quả
if missing_images:
    print("Các file label không có ảnh tương ứng:")
    for name in missing_images:
        print(f"- {name}.txt (không có {name}.jpg)")
else:
    print("Tất cả file label đều có ảnh tương ứng.")

if extra_images:
    print("Các file ảnh không có label tương ứng:")
    for name in extra_images:
        print(f"- {name}.jpg (không có {name}.txt)")
else:
    print("Tất cả file ảnh đều có label tương ứng.")

# Tùy chọn: In danh sách đầy đủ để kiểm tra
print("\nDanh sách file ảnh:", sorted(image_files))
print("Danh sách file label:", sorted(label_files))


Pretrain


In [2]:
# curl -o pretrain/en_PP-OCRv3_rec_train.tar https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_rec_train.tar


Download complete!


In [ ]:
import paddle
if paddle.device.is_compiled_with_cuda():
    print("GPU Name:", paddle.device.cuda.get_device_name(0))
    print("Current CUDA Device ID:", paddle.device.cuda.current_device())


In [ ]:
import torch
print("PyTorch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
print("cuDNN Available:", torch.backends.cudnn.is_available())


In [ ]:
import os

# Đường dẫn đến thư mục bạn muốn đếm
folder_path = r"D:\Private\Project\kaggle\OCR\PaddleOCR\data_OCR\labels"  # Thay bằng đường dẫn thực tế, ví dụ: "C:/Users/YourName/Documents"

# Đếm số lượng file
file_count = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])

print(f"Số lượng file trong thư mục: {file_count}")

In [ ]:
import matplotlib.pyplot as plt

# Dữ liệu lịch trình từ biểu đồ Gantt
schedule = [
    # (machine, job, operation, start_time, duration)
    (0, 1, 0, 0, 2),  # Machine 0: Job 1, operation 0
    (0, 0, 0, 2, 3),  # Machine 0: Job 0, operation 0
    (1, 2, 0, 0, 4),  # Machine 1: Job 2, operation 0
    (1, 1, 2, 4, 4),  # Machine 1: Job 1, operation 2
    (1, 0, 1, 8, 2),  # Machine 1: Job 0, operation 1
    (2, 1, 1, 2, 1),  # Machine 2: Job 1, operation 1
    (2, 2, 1, 4, 3),  # Machine 2: Job 2, operation 1
    (2, 0, 2, 10, 2), # Machine 2: Job 0, operation 2
]

# Màu sắc cho từng job
colors = {0: 'blue', 1: 'red', 2: 'green'}

# Tạo biểu đồ Gantt
fig, ax = plt.subplots(figsize=(10, 3))

for task in schedule:
    machine, job, operation, start, duration = task
    ax.broken_barh([(start, duration)], (machine - 0.4, 0.8), facecolors=colors[job], edgecolors='black')
    ax.text(start + duration / 2, machine, f'job({job},{operation})', ha='center', va='center', color='white')

# Cấu hình trục
ax.set_yticks(range(3))
ax.set_yticklabels(['Machine 0', 'Machine 1', 'Machine 2'])
ax.set_xticks(range(13))
ax.set_xlabel('Time')
ax.grid(True, axis='x')

plt.title('Gantt Chart for Job Shop Scheduling')
plt.show()

In [ ]:
from ortools.sat.python import cp_model

def solve_job_shop():
    # Tạo model
    model = cp_model.CpModel()

    # Dữ liệu ví dụ: [job_id, task_id, machine_id, duration]
    jobs_data = [
        [(0, 0, 0, 3), (0, 1, 1, 2), (0, 2, 2, 2)],  # Job 0
        [(1, 0, 1, 2), (1, 1, 2, 1), (1, 2, 0, 4)],  # Job 1
        [(2, 0, 2, 4), (2, 1, 0, 1), (2, 2, 1, 3)]   # Job 2
    ]

    machines_count = 3
    all_machines = range(machines_count)
    jobs_count = len(jobs_data)
    all_jobs = range(jobs_count)

    # Tính horizon (tổng thời gian tối đa)
    horizon = sum(task[3] for job in jobs_data for task in job)

    # Tạo biến
    task_vars = {}
    for job_id, job in enumerate(jobs_data):
        for task_id, task in enumerate(job):
            _, _, machine, duration = task
            suffix = f'_{job_id}_{task_id}'
            start_var = model.NewIntVar(0, horizon, 'start' + suffix)
            end_var = model.NewIntVar(0, horizon, 'end' + suffix)
            interval_var = model.NewIntervalVar(start_var, duration, end_var,
                                              'interval' + suffix)
            task_vars[(job_id, task_id)] = (start_var, end_var, interval_var, machine)

    # Thêm ràng buộc thứ tự cho các task trong mỗi job
    for job_id in all_jobs:
        for task_id in range(len(jobs_data[job_id]) - 1):
            model.Add(task_vars[(job_id, task_id)][1] <= 
                     task_vars[(job_id, task_id + 1)][0])

    # Ràng buộc không overlap trên từng máy
    machine_to_intervals = {machine: [] for machine in all_machines}
    for job_id, job in enumerate(jobs_data):
        for task_id, task in enumerate(job):
            machine = task[2]
            machine_to_intervals[machine].append(task_vars[(job_id, task_id)][2])

    for machine in all_machines:
        model.AddNoOverlap(machine_to_intervals[machine])

    # Hàm mục tiêu: tối thiểu makespan
    makespan = model.NewIntVar(0, horizon, 'makespan')
    for job_id in all_jobs:
        last_task_id = len(jobs_data[job_id]) - 1
        model.Add(makespan >= task_vars[(job_id, last_task_id)][1])
    model.Minimize(makespan)

    # Giải bài toán
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    # In kết quả
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print(f'Tổng thời gian hoàn thành tối ưu: {solver.Value(makespan)}')
        for job_id in all_jobs:
            print(f'Job {job_id}:')
            for task_id in range(len(jobs_data[job_id])):
                start = solver.Value(task_vars[(job_id, task_id)][0])
                duration = jobs_data[job_id][task_id][3]
                machine = jobs_data[job_id][task_id][2]
                print(f'  Task {task_id}: Machine {machine}, Start {start}, Duration {duration}')
    else:
        print('Không tìm thấy giải pháp.')

if __name__ == '__main__':
    solve_job_shop()

In [ ]:
from ortools.sat.python import cp_model
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def solve_and_plot_job_shop():
    # Tạo model
    model = cp_model.CpModel()

    # Dữ liệu ví dụ: [job_id, task_id, machine_id, duration]
    jobs_data = [
        [(0, 0, 0, 3), (0, 1, 1, 2), (0, 2, 2, 2)],  # Job 0
        [(1, 0, 1, 2), (1, 1, 2, 1), (1, 2, 0, 4)],  # Job 1
        [(2, 0, 2, 4), (2, 1, 0, 1), (2, 2, 1, 3)]   # Job 2
    ]

    machines_count = 3
    all_machines = range(machines_count)
    jobs_count = len(jobs_data)
    all_jobs = range(jobs_count)

    # Tính horizon
    horizon = sum(task[3] for job in jobs_data for task in job)

    # Tạo biến
    task_vars = {}
    for job_id, job in enumerate(jobs_data):
        for task_id, task in enumerate(job):
            _, _, machine, duration = task
            suffix = f'_{job_id}_{task_id}'
            start_var = model.NewIntVar(0, horizon, 'start' + suffix)
            end_var = model.NewIntVar(0, horizon, 'end' + suffix)
            interval_var = model.NewIntervalVar(start_var, duration, end_var,
                                              'interval' + suffix)
            task_vars[(job_id, task_id)] = (start_var, end_var, interval_var, machine)

    # Ràng buộc thứ tự task trong job
    for job_id in all_jobs:
        for task_id in range(len(jobs_data[job_id]) - 1):
            model.Add(task_vars[(job_id, task_id)][1] <= 
                     task_vars[(job_id, task_id + 1)][0])

    # Ràng buộc không overlap trên máy
    machine_to_intervals = {machine: [] for machine in all_machines}
    for job_id, job in enumerate(jobs_data):
        for task_id, task in enumerate(job):
            machine = task[2]
            machine_to_intervals[machine].append(task_vars[(job_id, task_id)][2])

    for machine in all_machines:
        model.AddNoOverlap(machine_to_intervals[machine])

    # Hàm mục tiêu: tối thiểu makespan
    makespan = model.NewIntVar(0, horizon, 'makespan')
    for job_id in all_jobs:
        last_task_id = len(jobs_data[job_id]) - 1
        model.Add(makespan >= task_vars[(job_id, last_task_id)][1])
    model.Minimize(makespan)

    # Giải bài toán
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    # Thu thập dữ liệu để vẽ
    schedule = []
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print(f'Tổng thời gian hoàn thành tối ưu: {solver.Value(makespan)}')
        for job_id in all_jobs:
            for task_id in range(len(jobs_data[job_id])):
                start = solver.Value(task_vars[(job_id, task_id)][0])
                duration = jobs_data[job_id][task_id][3]
                machine = jobs_data[job_id][task_id][2]
                schedule.append((job_id, task_id, machine, start, duration))
                print(f'Job {job_id}, Task {task_id}: Machine {machine}, Start {start}, Duration {duration}')
    else:
        print('Không tìm thấy giải pháp.')
        return

    # Vẽ biểu đồ Gantt
    fig, ax = plt.subplots(figsize=(10, 5))
    colors = ['#FF9999', '#99CCFF', '#99FF99']  # Màu cho từng Job

    for job_id, task_id, machine, start, duration in schedule:
        # Vẽ hình chữ nhật cho mỗi task
        ax.add_patch(patches.Rectangle(
            (start, machine - 0.4), duration, 0.8, 
            facecolor=colors[job_id], edgecolor='black',
            label=f'Job {job_id}' if task_id == 0 else ""
        ))
        # Ghi chú trên task
        ax.text(start + duration/2, machine, f'J{job_id}-T{task_id}', 
                ha='center', va='center', color='black')

    # Thiết lập biểu đồ
    ax.set_xlim(0, solver.Value(makespan) + 1)
    ax.set_ylim(-0.5, machines_count - 0.5)
    ax.set_yticks(range(machines_count))
    ax.set_yticklabels([f'Machine {m}' for m in all_machines])
    ax.set_xlabel('Thời gian')
    ax.set_title('Biểu đồ Gantt - Lịch trình Job Shop Scheduling')
    ax.grid(True, linestyle='--', alpha=0.7)

    # Thêm chú thích (legend) mà không lặp lại
    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys(), loc='upper right')

    plt.show()

if __name__ == '__main__':
    solve_and_plot_job_shop()

In [ ]:
import collections
from ortools.sat.python import cp_model
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def main():
    """Minimal jobshop problem with Gantt chart visualization."""
    # Data
    jobs_data = [  # task = (machine_id, processing_time)
        [(0, 3), (1, 2), (2, 2)],  # Job0
        [(0, 2), (2, 1), (1, 4)],  # Job1
        [(1, 4), (2, 3)],          # Job2
    ]

    machines_count = 1 + max(task[0] for job in jobs_data for task in job)
    all_machines = range(machines_count)
    horizon = sum(task[1] for job in jobs_data for task in job)

    # Create the model
    model = cp_model.CpModel()

    # Named tuple for variables
    task_type = collections.namedtuple("task_type", "start end interval")
    assigned_task_type = collections.namedtuple("assigned_task_type", "start job index duration")

    # Create job intervals and machine lists
    all_tasks = {}
    machine_to_intervals = collections.defaultdict(list)

    for job_id, job in enumerate(jobs_data):
        for task_id, task in enumerate(job):
            machine, duration = task
            suffix = f"_{job_id}_{task_id}"
            start_var = model.new_int_var(0, horizon, "start" + suffix)
            end_var = model.new_int_var(0, horizon, "end" + suffix)
            interval_var = model.new_interval_var(start_var, duration, end_var, "interval" + suffix)
            all_tasks[job_id, task_id] = task_type(start=start_var, end=end_var, interval=interval_var)
            machine_to_intervals[machine].append(interval_var)

    # Add disjunctive constraints (no overlap on machines)
    for machine in all_machines:
        model.add_no_overlap(machine_to_intervals[machine])

    # Add precedence constraints within jobs
    for job_id, job in enumerate(jobs_data):
        for task_id in range(len(job) - 1):
            model.add(all_tasks[job_id, task_id + 1].start >= all_tasks[job_id, task_id].end)

    # Makespan objective
    obj_var = model.new_int_var(0, horizon, "makespan")
    model.add_max_equality(obj_var, [all_tasks[job_id, len(job) - 1].end for job_id, job in enumerate(jobs_data)])
    model.minimize(obj_var)

    # Solve the model
    solver = cp_model.CpSolver()
    status = solver.solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print("Solution:")
        assigned_jobs = collections.defaultdict(list)
        schedule = []  # For Gantt chart

        # Collect assigned tasks
        for job_id, job in enumerate(jobs_data):
            for task_id, task in enumerate(job):
                machine = task[0]
                start = solver.value(all_tasks[job_id, task_id].start)
                duration = task[1]
                assigned_jobs[machine].append(
                    assigned_task_type(start=start, job=job_id, index=task_id, duration=duration)
                )
                schedule.append((job_id, task_id, machine, start, duration))

        # Print solution
        output = ""
        for machine in all_machines:
            assigned_jobs[machine].sort()
            sol_line_tasks = f"Machine {machine}: "
            sol_line = "           "
            for assigned_task in assigned_jobs[machine]:
                name = f"job_{assigned_task.job}_task_{assigned_task.index}"
                sol_line_tasks += f"{name:15}"
                sol_tmp = f"[{assigned_task.start},{assigned_task.start + assigned_task.duration}]"
                sol_line += f"{sol_tmp:15}"
            output += sol_line_tasks + "\n" + sol_line + "\n"
        
        print(f"Optimal Schedule Length: {solver.objective_value}")
        print(output)

        # Plot Gantt chart
        plot_gantt_chart(schedule, machines_count, solver.objective_value)
    else:
        print("No solution found.")

    # Statistics
    print("\nStatistics")
    print(f"  - conflicts: {solver.num_conflicts}")
    print(f"  - branches : {solver.num_branches}")
    print(f"  - wall time: {solver.wall_time}s")

def plot_gantt_chart(schedule, machines_count, makespan):
    """Plot Gantt chart using matplotlib."""
    fig, ax = plt.subplots(figsize=(10, 5))
    colors = ['#FF9999', '#99CCFF', '#99FF99']  # Colors for Job 0, 1, 2

    for job_id, task_id, machine, start, duration in schedule:
        ax.add_patch(patches.Rectangle(
            (start, machine - 0.4), duration, 0.8,
            facecolor=colors[job_id], edgecolor='black',
            label=f'Job {job_id}' if task_id == 0 else ""
        ))
        ax.text(start + duration/2, machine, f'J{job_id}-T{task_id}',
                ha='center', va='center', color='black')

    # Chart settings
    ax.set_xlim(0, makespan + 1)
    ax.set_ylim(-0.5, machines_count - 0.5)
    ax.set_yticks(range(machines_count))
    ax.set_yticklabels([f'Machine {m}' for m in range(machines_count)])
    ax.set_xlabel('Thời gian')
    ax.set_title('Biểu đồ Gantt - Lịch trình Job Shop Scheduling')
    ax.grid(True, linestyle='--', alpha=0.7)

    # Add legend without duplicates
    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys(), loc='upper right')

    plt.show()

if __name__ == "__main__":
    main()

In [ ]:
import collections
from ortools.sat.python import cp_model
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def main():
    """Minimal jobshop problem with Gantt chart visualization."""
    # Data
    jobs_data = [  # task = (machine_id, processing_time)
        [(0, 3), (1, 2), (2, 2)],  # Job0
        [(0, 2), (2, 1), (1, 4)],  # Job1
        [(1, 4), (2, 3)],          # Job2
    ]

    machines_count = 1 + max(task[0] for job in jobs_data for task in job)
    all_machines = range(machines_count)
    horizon = sum(task[1] for job in jobs_data for task in job)

    # Create the model
    model = cp_model.CpModel()

    # Named tuple for variables
    task_type = collections.namedtuple("task_type", "start end interval")
    assigned_task_type = collections.namedtuple("assigned_task_type", "start job index duration")

    # Create job intervals and machine lists
    all_tasks = {}
    machine_to_intervals = collections.defaultdict(list)

    for job_id, job in enumerate(jobs_data):
        for task_id, task in enumerate(job):
            machine, duration = task
            suffix = f"_{job_id}_{task_id}"
            start_var = model.new_int_var(0, horizon, "start" + suffix)
            end_var = model.new_int_var(0, horizon, "end" + suffix)
            interval_var = model.new_interval_var(start_var, duration, end_var, "interval" + suffix)
            all_tasks[job_id, task_id] = task_type(start=start_var, end=end_var, interval=interval_var)
            machine_to_intervals[machine].append(interval_var)

    # Add disjunctive constraints (no overlap on machines)
    for machine in all_machines:
        model.add_no_overlap(machine_to_intervals[machine])

    # Add precedence constraints within jobs
    for job_id, job in enumerate(jobs_data):
        for task_id in range(len(job) - 1):
            model.add(all_tasks[job_id, task_id + 1].start >= all_tasks[job_id, task_id].end)

    # Makespan objective
    obj_var = model.new_int_var(0, horizon, "makespan")
    model.add_max_equality(obj_var, [all_tasks[job_id, len(job) - 1].end for job_id, job in enumerate(jobs_data)])
    model.minimize(obj_var)

    # Solve the model
    solver = cp_model.CpSolver()
    status = solver.solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print("Solution:")
        assigned_jobs = collections.defaultdict(list)
        schedule = []  # For Gantt chart

        # Collect assigned tasks
        for job_id, job in enumerate(jobs_data):
            for task_id, task in enumerate(job):
                machine = task[0]
                start = solver.value(all_tasks[job_id, task_id].start)
                duration = task[1]
                assigned_jobs[machine].append(
                    assigned_task_type(start=start, job=job_id, index=task_id, duration=duration)
                )
                schedule.append((job_id, task_id, machine, start, duration))

        # Print solution
        output = ""
        for machine in all_machines:
            assigned_jobs[machine].sort()
            sol_line_tasks = f"Machine {machine}: "
            sol_line = "           "
            for assigned_task in assigned_jobs[machine]:
                name = f"job_{assigned_task.job}_task_{assigned_task.index}"
                sol_line_tasks += f"{name:15}"
                sol_tmp = f"[{assigned_task.start},{assigned_task.start + assigned_task.duration}]"
                sol_line += f"{sol_tmp:15}"
            output += sol_line_tasks + "\n" + sol_line + "\n"
        
        print(f"Optimal Schedule Length: {solver.objective_value}")
        print(output)

        # Plot Gantt chart with clear time labels
        plot_gantt_chart(schedule, machines_count, solver.objective_value)
    else:
        print("No solution found.")

    # Statistics
    print("\nStatistics")
    print(f"  - conflicts: {solver.num_conflicts}")
    print(f"  - branches : {solver.num_branches}")
    print(f"  - wall time: {solver.wall_time}s")

def plot_gantt_chart(schedule, machines_count, makespan):
    """Plot Gantt chart with clear time labels using matplotlib."""
    fig, ax = plt.subplots(figsize=(12, 5))  # Tăng kích thước để rõ hơn
    colors = ['#FF9999', '#99CCFF', '#99FF99']  # Colors for Job 0, 1, 2

    for job_id, task_id, machine, start, duration in schedule:
        ax.add_patch(patches.Rectangle(
            (start, machine - 0.4), duration, 0.8,
            facecolor=colors[job_id], edgecolor='black',
            label=f'Job {job_id}' if task_id == 0 else ""
        ))
        ax.text(start + duration/2, machine, f'J{job_id}-T{task_id}',
                ha='center', va='center', color='black', fontsize=10)

    # Chart settings with clear time ticks
    ax.set_xlim(0, makespan + 1)
    ax.set_ylim(-0.5, machines_count - 0.5)
    ax.set_yticks(range(machines_count))
    ax.set_yticklabels([f'Machine {m}' for m in range(machines_count)])

    # Thêm mốc thời gian chi tiết trên trục x
    time_ticks = range(0, int(makespan) + 1, 1)  # Mỗi đơn vị thời gian là 1
    ax.set_xticks(time_ticks)
    ax.set_xticklabels([str(t) for t in time_ticks], rotation=45, ha='right')  # Xoay nhãn cho dễ đọc
    ax.set_xlabel('Thời gian')
    ax.set_title('Biểu đồ Gantt - Lịch trình Job Shop Scheduling')
    ax.grid(True, linestyle='--', alpha=0.7)

    # Add legend without duplicates
    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys(), loc='upper right')

    plt.tight_layout()  # Tự động điều chỉnh khoảng cách để tránh chồng lấn
    plt.show()

if __name__ == "__main__":
    main()

In [ ]:
import collections
from ortools.sat.python import cp_model
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def main():
    """Minimal jobshop problem with Gantt chart visualization."""
    # Data
    jobs_data = [  # task = (machine_id, processing_time)
        [(0, 3), (1, 2), (2, 2)],  # Job0
        [(0, 2), (2, 1), (1, 4)],  # Job1
        [(1, 4), (2, 3)],          # Job2
    ]

    machines_count = 1 + max(task[0] for job in jobs_data for task in job)
    all_machines = range(machines_count)
    horizon = sum(task[1] for job in jobs_data for task in job)

    # Create the model
    model = cp_model.CpModel()

    # Named tuple for variables
    task_type = collections.namedtuple("task_type", "start end interval")
    assigned_task_type = collections.namedtuple("assigned_task_type", "start job index duration")

    # Create job intervals and machine lists
    all_tasks = {}
    machine_to_intervals = collections.defaultdict(list)

    for job_id, job in enumerate(jobs_data):
        for task_id, task in enumerate(job):
            machine, duration = task
            suffix = f"_{job_id}_{task_id}"
            start_var = model.new_int_var(0, horizon, "start" + suffix)
            end_var = model.new_int_var(0, horizon, "end" + suffix)
            interval_var = model.new_interval_var(start_var, duration, end_var, "interval" + suffix)
            all_tasks[job_id, task_id] = task_type(start=start_var, end=end_var, interval=interval_var)
            machine_to_intervals[machine].append(interval_var)

    # Add disjunctive constraints (no overlap on machines)
    for machine in all_machines:
        model.add_no_overlap(machine_to_intervals[machine])

    # Add precedence constraints within jobs
    for job_id, job in enumerate(jobs_data):
        for task_id in range(len(job) - 1):
            model.add(all_tasks[job_id, task_id + 1].start >= all_tasks[job_id, task_id].end)

    # Makespan objective
    obj_var = model.new_int_var(0, horizon, "makespan")
    model.add_max_equality(obj_var, [all_tasks[job_id, len(job) - 1].end for job_id, job in enumerate(jobs_data)])
    model.minimize(obj_var)

    # Solve the model
    solver = cp_model.CpSolver()
    status = solver.solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print("Solution:")
        assigned_jobs = collections.defaultdict(list)
        schedule = []  # For Gantt chart

        # Collect assigned tasks
        for job_id, job in enumerate(jobs_data):
            for task_id, task in enumerate(job):
                machine = task[0]
                start = solver.value(all_tasks[job_id, task_id].start)
                duration = task[1]
                assigned_jobs[machine].append(
                    assigned_task_type(start=start, job=job_id, index=task_id, duration=duration)
                )
                schedule.append((job_id, task_id, machine, start, duration))

        # Print solution
        output = ""
        for machine in all_machines:
            assigned_jobs[machine].sort()
            sol_line_tasks = f"Machine {machine}: "
            sol_line = "           "
            for assigned_task in assigned_jobs[machine]:
                name = f"job_{assigned_task.job}_task_{assigned_task.index}"
                sol_line_tasks += f"{name:15}"
                sol_tmp = f"[{assigned_task.start},{assigned_task.start + assigned_task.duration}]"
                sol_line += f"{sol_tmp:15}"
            output += sol_line_tasks + "\n" + sol_line + "\n"
        
        print(f"Optimal Schedule Length: {solver.objective_value}")
        print(output)

        # Plot Gantt chart with clear time labels
        plot_gantt_chart(schedule, machines_count, solver.objective_value)
    else:
        print("No solution found.")

    # Statistics
    print("\nStatistics")
    print(f"  - conflicts: {solver.num_conflicts}")
    print(f"  - branches : {solver.num_branches}")
    print(f"  - wall time: {solver.wall_time}s")

def plot_gantt_chart(schedule, machines_count, makespan):
    """Plot Gantt chart with straight time labels using matplotlib."""
    fig, ax = plt.subplots(figsize=(12, 5))  # Kích thước biểu đồ
    colors = ['#FF9999', '#99CCFF', '#99FF99']  # Colors for Job 0, 1, 2

    for job_id, task_id, machine, start, duration in schedule:
        ax.add_patch(patches.Rectangle(
            (start, machine - 0.4), duration, 0.8,
            facecolor=colors[job_id], edgecolor='black',
            label=f'Job {job_id}' if task_id == 0 else ""
        ))
        ax.text(start + duration/2, machine, f'J{job_id}-T{task_id}',
                ha='center', va='center', color='black', fontsize=10)

    # Chart settings with clear and straight time ticks
    ax.set_xlim(0, makespan + 1)
    ax.set_ylim(-0.5, machines_count - 0.5)
    ax.set_yticks(range(machines_count))
    ax.set_yticklabels([f'Machine {m}' for m in range(machines_count)])

    # Thêm mốc thời gian chi tiết trên trục x, giữ thẳng
    time_ticks = range(0, int(makespan) + 1, 1)  # Mỗi đơn vị thời gian là 1
    ax.set_xticks(time_ticks)
    ax.set_xticklabels([str(t) for t in time_ticks], ha='center')  # Giữ thẳng, căn giữa
    ax.set_xlabel('Thời gian')
    ax.grid(True, linestyle='--', alpha=0.7)

    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys(), loc='upper right')

    plt.tight_layout()  
    plt.show()

if __name__ == "__main__":
    main()

In [ ]:
import collections
from ortools.sat.python import cp_model
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def main():
    """Job Shop Scheduling with Gantt chart visualization."""
    # Data: task = (machine_id, processing_time)
    jobs_data = [
        [(0, 3), (1, 2), (2, 2)],  # J0: O1(0,3), O2(1,2), O3(2,2)
        [(0, 2), (2, 1), (1, 4)],  # J1: O1(0,2), O2(2,1), O3(1,4)
        [(1, 4), (2, 3)],          # J2: O1(1,4), O2(2,3), (bỏ qua O3)
    ]

    machines_count = 1 + max(task[0] for job in jobs_data for task in job)
    all_machines = range(machines_count)
    horizon = sum(task[1] for job in jobs_data for task in job)

    # Create the model
    model = cp_model.CpModel()

    # Named tuple for variables
    task_type = collections.namedtuple("task_type", "start end interval")
    assigned_task_type = collections.namedtuple("assigned_task_type", "start job index duration")

    # Create job intervals and machine lists
    all_tasks = {}
    machine_to_intervals = collections.defaultdict(list)

    for job_id, job in enumerate(jobs_data):
        for task_id, task in enumerate(job):
            machine, duration = task
            if duration == 0:  # Bỏ qua task có duration = 0 (như O3 của J2)
                continue
            suffix = f"_{job_id}_{task_id}"
            start_var = model.new_int_var(0, horizon, "start" + suffix)
            end_var = model.new_int_var(0, horizon, "end" + suffix)
            interval_var = model.new_interval_var(start_var, duration, end_var, "interval" + suffix)
            all_tasks[job_id, task_id] = task_type(start=start_var, end=end_var, interval=interval_var)
            machine_to_intervals[machine].append(interval_var)

    # Add disjunctive constraints (no overlap on machines)
    for machine in all_machines:
        model.add_no_overlap(machine_to_intervals[machine])

    # Add precedence constraints within jobs
    for job_id, job in enumerate(jobs_data):
        for task_id in range(len(job) - 1):
            if (job_id, task_id) in all_tasks and (job_id, task_id + 1) in all_tasks:  # Chỉ thêm nếu task tồn tại
                model.add(all_tasks[job_id, task_id + 1].start >= all_tasks[job_id, task_id].end)

    # Makespan objective
    obj_var = model.new_int_var(0, horizon, "makespan")
    model.add_max_equality(obj_var, [all_tasks[job_id, len(job) - 1].end 
                                    for job_id, job in enumerate(jobs_data) 
                                    if (job_id, len(job) - 1) in all_tasks])
    model.minimize(obj_var)

    # Solve the model
    solver = cp_model.CpSolver()
    status = solver.solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print("Solution:")
        assigned_jobs = collections.defaultdict(list)
        schedule = []  # For Gantt chart

        # Collect assigned tasks
        for job_id, job in enumerate(jobs_data):
            for task_id, task in enumerate(job):
                machine, duration = task
                if duration == 0:
                    continue
                start = solver.value(all_tasks[job_id, task_id].start)
                assigned_jobs[machine].append(
                    assigned_task_type(start=start, job=job_id, index=task_id, duration=duration)
                )
                schedule.append((job_id, task_id, machine, start, duration))

        # Print solution
        output = ""
        for machine in all_machines:
            assigned_jobs[machine].sort()
            sol_line_tasks = f"Machine {machine}: "
            sol_line = "           "
            for assigned_task in assigned_jobs[machine]:
                name = f"J{assigned_task.job}-O{assigned_task.index + 1}"
                sol_line_tasks += f"{name:15}"
                sol_tmp = f"[{assigned_task.start},{assigned_task.start + assigned_task.duration}]"
                sol_line += f"{sol_tmp:15}"
            output += sol_line_tasks + "\n" + sol_line + "\n"
        
        print(f"Optimal Schedule Length: {solver.objective_value}")
        print(output)

        # Plot Gantt chart
        plot_gantt_chart(schedule, machines_count, solver.objective_value)
    else:
        print("No solution found.")

def plot_gantt_chart(schedule, machines_count, makespan):
    """Plot Gantt chart with J and O labels."""
    fig, ax = plt.subplots(figsize=(12, 5))
    colors = ['#FF9999', '#99CCFF', '#99FF99']  # Colors for J0, J1, J2

    for job_id, task_id, machine, start, duration in schedule:
        ax.add_patch(patches.Rectangle(
            (start, machine - 0.4), duration, 0.8,
            facecolor=colors[job_id], edgecolor='black',
            label=f'J{job_id}' if task_id == 0 else ""
        ))
        # Hiển thị J và O trên task
        ax.text(start + duration/2, machine, f'J{job_id}-O{task_id + 1}',
                ha='center', va='center', color='black', fontsize=20)

    # Chart settings with clear time ticks
    ax.set_xlim(0, makespan + 1)
    ax.set_ylim(-0.5, machines_count - 0.5)
    
    ax.set_yticks(range(machines_count))
    ax.set_yticklabels([f'Machine {m}' for m in range(machines_count)], fontsize=20, fontweight='bold')


    # Thêm mốc thời gian chi tiết trên trục x
    time_ticks = range(0, int(makespan) + 1, 1)
    ax.set_xticks(time_ticks)
    ax.set_xticklabels([str(t) for t in time_ticks], ha='center')
    ax.set_xlabel('Thời gian', fontsize=20, fontweight='bold')
    ax.grid(True, linestyle='--', alpha=0.7)

    # Add legend without duplicates
    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys(), loc='upper right')

    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    main()

In [ ]:
import collections
import matplotlib.pyplot as plt
from ortools.sat.python import cp_model


def main():
    """Minimal jobshop problem with Gantt chart."""
    # Data.
    jobs_data = [  # task = (machine_id, processing_time).
        [(0, 3), (1, 2), (2, 2)],  # Job0
        [(0, 2), (2, 1), (1, 4)],  # Job1
        [(1, 4), (2, 3)],  # Job2
    ]

    machines_count = 1 + max(task[0] for job in jobs_data for task in job)
    all_machines = range(machines_count)
    horizon = sum(task[1] for job in jobs_data for task in job)

    # Create the model.
    model = cp_model.CpModel()
    task_type = collections.namedtuple("task_type", "start end interval")
    assigned_task_type = collections.namedtuple(
        "assigned_task_type", "start job index duration"
    )

    all_tasks = {}
    machine_to_intervals = collections.defaultdict(list)

    for job_id, job in enumerate(jobs_data):
        for task_id, task in enumerate(job):
            machine, duration = task
            suffix = f"_{job_id}_{task_id}"
            start_var = model.new_int_var(0, horizon, "start" + suffix)
            end_var = model.new_int_var(0, horizon, "end" + suffix)
            interval_var = model.new_interval_var(
                start_var, duration, end_var, "interval" + suffix
            )
            all_tasks[job_id, task_id] = task_type(
                start=start_var, end=end_var, interval=interval_var
            )
            machine_to_intervals[machine].append(interval_var)

    for machine in all_machines:
        model.add_no_overlap(machine_to_intervals[machine])

    for job_id, job in enumerate(jobs_data):
        for task_id in range(len(job) - 1):
            model.add(
                all_tasks[job_id, task_id + 1].start >= all_tasks[job_id, task_id].end
            )

    obj_var = model.new_int_var(0, horizon, "makespan")
    model.add_max_equality(
        obj_var,
        [all_tasks[job_id, len(job) - 1].end for job_id, job in enumerate(jobs_data)],
    )
    model.minimize(obj_var)

    solver = cp_model.CpSolver()
    status = solver.solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print("Solution:")
        assigned_jobs = collections.defaultdict(list)
        for job_id, job in enumerate(jobs_data):
            for task_id, task in enumerate(job):
                machine = task[0]
                assigned_jobs[machine].append(
                    assigned_task_type(
                        start=solver.value(all_tasks[job_id, task_id].start),
                        job=job_id,
                        index=task_id,
                        duration=task[1],
                    )
                )

        output = ""
        for machine in all_machines:
            assigned_jobs[machine].sort()
            sol_line_tasks = "Machine " + str(machine) + ": "
            sol_line = "           "
            for assigned_task in assigned_jobs[machine]:
                name = f"job_{assigned_task.job}_task_{assigned_task.index}"
                sol_line_tasks += f"{name:15}"
                start = assigned_task.start
                duration = assigned_task.duration
                sol_tmp = f"[{start},{start + duration}]"
                sol_line += f"{sol_tmp:15}"
            sol_line += "\n"
            sol_line_tasks += "\n"
            output += sol_line_tasks
            output += sol_line

        print(f"Optimal Schedule Length: {solver.objective_value}")
        print(output)

        # Vẽ biểu đồ Gantt
        fig, ax = plt.subplots(figsize=(10, 5))
        colors = ["red", "blue", "green", "orange", "purple"]
        for machine in all_machines:
            for assigned_task in assigned_jobs[machine]:
                start = assigned_task.start
                duration = assigned_task.duration
                job_id = assigned_task.job
                task_label = f"J{job_id}-T{assigned_task.index}"
                ax.broken_barh([(start, duration)], (machine - 0.4, 0.8),
                               facecolors=colors[job_id % len(colors)], label=task_label)

        ax.set_xlabel("Time")
        ax.set_ylabel("Machines")
        ax.set_yticks(list(all_machines))
        ax.set_yticklabels([f"Machine {m}" for m in all_machines])
        ax.grid(True)

        plt.title("Gantt Chart for Job Shop Scheduling")
        plt.show()
    else:
        print("No solution found.")

    print("\nStatistics")
    print(f"  - conflicts: {solver.num_conflicts}")
    print(f"  - branches : {solver.num_branches}")
    print(f"  - wall time: {solver.wall_time}s")


if __name__ == "__main__":
    main()


In [ ]:
"""Minimal jobshop example."""
import collections
from ortools.sat.python import cp_model


def main() -> None:
    """Minimal jobshop problem."""
    # Data.
    jobs_data = [  # task = (machine_id, processing_time).
        [(0, 3), (1, 2), (2, 2)],  # Job0
        [(0, 2), (2, 1), (1, 4)],  # Job1
        [(1, 4), (2, 3)],  # Job2
    ]

    machines_count = 1 + max(task[0] for job in jobs_data for task in job)
    all_machines = range(machines_count)
    # Computes horizon dynamically as the sum of all durations.
    horizon = sum(task[1] for job in jobs_data for task in job)

    # Create the model.
    model = cp_model.CpModel()

    # Named tuple to store information about created variables.
    task_type = collections.namedtuple("task_type", "start end interval")
    # Named tuple to manipulate solution information.
    assigned_task_type = collections.namedtuple(
        "assigned_task_type", "start job index duration"
    )

    # Creates job intervals and add to the corresponding machine lists.
    all_tasks = {}
    machine_to_intervals = collections.defaultdict(list)

    for job_id, job in enumerate(jobs_data):
        for task_id, task in enumerate(job):
            machine, duration = task
            suffix = f"_{job_id}_{task_id}"
            start_var = model.new_int_var(0, horizon, "start" + suffix)
            end_var = model.new_int_var(0, horizon, "end" + suffix)
            interval_var = model.new_interval_var(
                start_var, duration, end_var, "interval" + suffix
            )
            all_tasks[job_id, task_id] = task_type(
                start=start_var, end=end_var, interval=interval_var
            )
            machine_to_intervals[machine].append(interval_var)

    # Create and add disjunctive constraints.
    for machine in all_machines:
        model.add_no_overlap(machine_to_intervals[machine])

    # Precedences inside a job.
    for job_id, job in enumerate(jobs_data):
        for task_id in range(len(job) - 1):
            model.add(
                all_tasks[job_id, task_id + 1].start >= all_tasks[job_id, task_id].end
            )

    # Makespan objective.
    obj_var = model.new_int_var(0, horizon, "makespan")
    model.add_max_equality(
        obj_var,
        [all_tasks[job_id, len(job) - 1].end for job_id, job in enumerate(jobs_data)],
    )
    model.minimize(obj_var)

    # Creates the solver and solve.
    solver = cp_model.CpSolver()
    status = solver.solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print("Solution:")
        # Create one list of assigned tasks per machine.
        assigned_jobs = collections.defaultdict(list)
        for job_id, job in enumerate(jobs_data):
            for task_id, task in enumerate(job):
                machine = task[0]
                assigned_jobs[machine].append(
                    assigned_task_type(
                        start=solver.value(all_tasks[job_id, task_id].start),
                        job=job_id,
                        index=task_id,
                        duration=task[1],
                    )
                )

        # Create per machine output lines.
        output = ""
        for machine in all_machines:
            # Sort by starting time.
            assigned_jobs[machine].sort()
            sol_line_tasks = "Machine " + str(machine) + ": "
            sol_line = "           "

            for assigned_task in assigned_jobs[machine]:
                name = f"job_{assigned_task.job}_task_{assigned_task.index}"
                # add spaces to output to align columns.
                sol_line_tasks += f"{name:15}"

                start = assigned_task.start
                duration = assigned_task.duration
                sol_tmp = f"[{start},{start + duration}]"
                # add spaces to output to align columns.
                sol_line += f"{sol_tmp:15}"

            sol_line += "\n"
            sol_line_tasks += "\n"
            output += sol_line_tasks
            output += sol_line

        # Finally print the solution found.
        print(f"Optimal Schedule Length: {solver.objective_value}")
        print(output)
    else:
        print("No solution found.")

    # Statistics.
    print("\nStatistics")
    print(f"  - conflicts: {solver.num_conflicts}")
    print(f"  - branches : {solver.num_branches}")
    print(f"  - wall time: {solver.wall_time}s")


if __name__ == "__main__":
    main()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Danh sách máy và công việc
machines = ["Machine 0", "Machine 1", "Machine 2"]
tasks = [
    (0, 0, 2, "J0-O1"), (2, 2, 3, "J0-O2"), (5, 0, 2, "J0-O3"),
    (0, 0, 2, "J1-O1"), (2, 2, 1, "J1-O2"), (3, 1, 2, "J1-O3"),
    (0, 2, 2, "J2-O1"), (2, 0, 3, "J2-O2"), (5, 1, 1, "J2-O3")
]

fig, ax = plt.subplots(figsize=(10, 5))

for start, machine, duration, label in tasks:
    ax.add_patch(mpatches.Rectangle((start, machine), duration, 0.8, facecolor="lightblue", edgecolor="black"))
    ax.text(start + duration / 2, machine + 0.4, label, ha='center', va='center', fontsize=10, color='black')

ax.set_yticks(range(len(machines)))
ax.set_yticklabels(machines)
ax.set_xlabel("Thời gian")
ax.set_title("Biểu đồ Gantt cho FJSS")

plt.grid(axis="x", linestyle="--", alpha=0.7)
plt.show()


In [ ]:
from ortools.sat.python import cp_model

def solve_fjss():
    
    jobs_data = [
        [[(0, 2), (1, 3)], [(1, 1), (2, 4)], [(0, 3), (2, 2)]],
        [[(1, 3), (2, 2)], [(0, 4), (1, 2)], [(1, 2), (2, 3)]],
        [[(0, 4), (2, 1)], [(1, 3), (2, 2)], [(0, 2), (1, 4)]],
    ]

    num_jobs = len(jobs_data)
    num_machines = 3
    all_machines = range(num_machines)

    # Khởi tạo mô hình
    model = cp_model.CpModel()

    # Tính thời gian tối đa có thể xảy ra (để làm giới hạn trên cho biến)
    horizon = sum(task[1] for job in jobs_data for operation in job for task in operation)
    
    # Tạo biến: (job_id, operation_id, machine_id) -> (start, duration, end)
    task_to_interval = {}
    task_to_machine = {}
    for job_id in range(num_jobs):
        for op_id in range(len(jobs_data[job_id])):
            for task_id, (machine_id, duration) in enumerate(jobs_data[job_id][op_id]):
                suffix = f'_j{job_id}_o{op_id}_m{machine_id}'
                start_var = model.NewIntVar(0, horizon, 'start' + suffix)
                end_var = model.NewIntVar(0, horizon, 'end' + suffix)
                interval_var = model.NewIntervalVar(start_var, duration, end_var, 'interval' + suffix)
                task_to_interval[(job_id, op_id, machine_id)] = interval_var
                task_to_machine[(job_id, op_id, machine_id)] = machine_id

    # Tạo biến để chọn máy cho mỗi thao tác
    machine_selection = {}
    for job_id in range(num_jobs):
        for op_id in range(len(jobs_data[job_id])):
            tasks = jobs_data[job_id][op_id]
            task_literals = []
            for task_id, (machine_id, _) in enumerate(tasks):
                literal = model.NewBoolVar(f'j{job_id}_o{op_id}_m{machine_id}')
                task_literals.append(literal)
                machine_selection[(job_id, op_id, machine_id)] = literal
            # Chỉ một máy được chọn cho mỗi thao tác
            model.AddExactlyOne(task_literals)

    # Ràng buộc: Các thao tác trong cùng một công việc phải thực hiện theo thứ tự
    for job_id in range(num_jobs):
        for op_id in range(len(jobs_data[job_id]) - 1):
            possible_machines_curr = jobs_data[job_id][op_id]
            possible_machines_next = jobs_data[job_id][op_id + 1]
            for _, (machine_curr, _) in enumerate(possible_machines_curr):
                for _, (machine_next, _) in enumerate(possible_machines_next):
                    curr_end = task_to_interval[(job_id, op_id, machine_curr)].EndExpr()
                    next_start = task_to_interval[(job_id, op_id + 1, machine_next)].StartExpr()
                    model.Add(curr_end <= next_start).OnlyEnforceIf([
                        machine_selection[(job_id, op_id, machine_curr)],
                        machine_selection[(job_id, op_id + 1, machine_next)]
                    ])

    # Ràng buộc: Không có hai thao tác nào trên cùng một máy được chồng lấp
    for machine_id in all_machines:
        intervals = []
        for job_id in range(num_jobs):
            for op_id in range(len(jobs_data[job_id])):
                for task_id, (m_id, _) in enumerate(jobs_data[job_id][op_id]):
                    if m_id == machine_id:
                        interval = task_to_interval[(job_id, op_id, m_id)]
                        intervals.append(interval)
                        model.Add(interval.StartExpr() >= 0).OnlyEnforceIf(machine_selection[(job_id, op_id, m_id)])
        model.AddNoOverlap(intervals)

    # Mục tiêu: Tối thiểu hóa thời gian hoàn thành tối đa (C_max)
    c_max = model.NewIntVar(0, horizon, 'c_max')
    for job_id in range(num_jobs):
        last_op_id = len(jobs_data[job_id]) - 1
        possible_machines = jobs_data[job_id][last_op_id]
        for _, (machine_id, _) in enumerate(possible_machines):
            end_time = task_to_interval[(job_id, last_op_id, machine_id)].EndExpr()
            model.Add(c_max >= end_time).OnlyEnforceIf(machine_selection[(job_id, last_op_id, machine_id)])
    model.Minimize(c_max)

    # Giải bài toán
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    # In kết quả
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print(f'Tổng thời gian hoàn thành tối đa (C_max): {solver.ObjectiveValue()}')
        print('\nLịch trình:')
        for job_id in range(num_jobs):
            print(f'Công việc {job_id}:')
            for op_id in range(len(jobs_data[job_id])):
                for task_id, (machine_id, duration) in enumerate(jobs_data[job_id][op_id]):
                    if solver.Value(machine_selection[(job_id, op_id, machine_id)]) == 1:
                        start = solver.Value(task_to_interval[(job_id, op_id, machine_id)].StartExpr())
                        end = solver.Value(task_to_interval[(job_id, op_id, machine_id)].EndExpr())
                        print(f'  Thao tác {op_id} trên máy {machine_id}: bắt đầu {start}, kết thúc {end}, thời gian xử lý {duration}')
    else:
        print('Không tìm thấy lịch trình khả thi.')

# Chạy chương trình
solve_fjss()

In [ ]:
from ortools.sat.python import cp_model

def solve_fjss():
    # Dữ liệu đầu vào
    jobs_data = [
        # Job 0
        [[(0, 3), (1, 2)], [(1, 2), (2, 3)], [(0, 2), (2, 2)]],
        # Job 1
        [[(0, 2), (2, 4)], [(1, 3), (2, 1)], [(0, 4), (1, 2)]],
        # Job 2
        [[(1, 4), (2, 2)], [(0, 3), (2, 3)], [(1, 1), (2, 3)]],
    ]

    num_jobs = len(jobs_data)
    num_machines = 3
    all_machines = range(num_machines)

    # Khởi tạo mô hình
    model = cp_model.CpModel()

    # Tính thời gian tối đa có thể xảy ra (để làm giới hạn trên cho biến)
    horizon = sum(task[1] for job in jobs_data for operation in job for task in operation)
    
    # Tạo biến: (job_id, operation_id, machine_id) -> (start, duration, end)
    task_to_interval = {}
    task_to_machine = {}
    for job_id in range(num_jobs):
        for op_id in range(len(jobs_data[job_id])):
            for task_id, (machine_id, duration) in enumerate(jobs_data[job_id][op_id]):
                suffix = f'_j{job_id}_o{op_id}_m{machine_id}'
                start_var = model.NewIntVar(0, horizon, 'start' + suffix)
                end_var = model.NewIntVar(0, horizon, 'end' + suffix)
                interval_var = model.NewIntervalVar(start_var, duration, end_var, 'interval' + suffix)
                task_to_interval[(job_id, op_id, machine_id)] = interval_var
                task_to_machine[(job_id, op_id, machine_id)] = machine_id

    # Tạo biến để chọn máy cho mỗi thao tác
    machine_selection = {}
    for job_id in range(num_jobs):
        for op_id in range(len(jobs_data[job_id])):
            tasks = jobs_data[job_id][op_id]
            task_literals = []
            for task_id, (machine_id, _) in enumerate(tasks):
                literal = model.NewBoolVar(f'j{job_id}_o{op_id}_m{machine_id}')
                task_literals.append(literal)
                machine_selection[(job_id, op_id, machine_id)] = literal
            # Chỉ một máy được chọn cho mỗi thao tác
            model.AddExactlyOne(task_literals)

    # Ràng buộc: Các thao tác trong cùng một công việc phải thực hiện theo thứ tự
    for job_id in range(num_jobs):
        for op_id in range(len(jobs_data[job_id]) - 1):
            possible_machines_curr = jobs_data[job_id][op_id]
            possible_machines_next = jobs_data[job_id][op_id + 1]
            for _, (machine_curr, _) in enumerate(possible_machines_curr):
                for _, (machine_next, _) in enumerate(possible_machines_next):
                    curr_end = task_to_interval[(job_id, op_id, machine_curr)].EndExpr()
                    next_start = task_to_interval[(job_id, op_id + 1, machine_next)].StartExpr()
                    model.Add(curr_end <= next_start).OnlyEnforceIf([
                        machine_selection[(job_id, op_id, machine_curr)],
                        machine_selection[(job_id, op_id + 1, machine_next)]
                    ])

    # Ràng buộc: Không có hai thao tác nào trên cùng một máy được chồng lấp
    for machine_id in all_machines:
        intervals = []
        for job_id in range(num_jobs):
            for op_id in range(len(jobs_data[job_id])):
                for task_id, (m_id, _) in enumerate(jobs_data[job_id][op_id]):
                    if m_id == machine_id:
                        interval = task_to_interval[(job_id, op_id, m_id)]
                        intervals.append(interval)
                        model.Add(interval.StartExpr() >= 0).OnlyEnforceIf(machine_selection[(job_id, op_id, m_id)])
        model.AddNoOverlap(intervals)

    # Mục tiêu: Tối thiểu hóa thời gian hoàn thành tối đa (C_max)
    c_max = model.NewIntVar(0, horizon, 'c_max')
    for job_id in range(num_jobs):
        last_op_id = len(jobs_data[job_id]) - 1
        possible_machines = jobs_data[job_id][last_op_id]
        for _, (machine_id, _) in enumerate(possible_machines):
            end_time = task_to_interval[(job_id, last_op_id, machine_id)].EndExpr()
            model.Add(c_max >= end_time).OnlyEnforceIf(machine_selection[(job_id, last_op_id, machine_id)])
    model.Minimize(c_max)

    # Giải bài toán
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    # In kết quả
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print(f'Tổng thời gian hoàn thành tối đa (C_max): {solver.ObjectiveValue()}')
        print('\nLịch trình:')
        schedule_data = []
        for job_id in range(num_jobs):
            print(f'Công việc {job_id}:')
            for op_id in range(len(jobs_data[job_id])):
                for task_id, (machine_id, duration) in enumerate(jobs_data[job_id][op_id]):
                    if solver.Value(machine_selection[(job_id, op_id, machine_id)]) == 1:
                        start = solver.Value(task_to_interval[(job_id, op_id, machine_id)].StartExpr())
                        end = solver.Value(task_to_interval[(job_id, op_id, machine_id)].EndExpr())
                        schedule_data.append((job_id, op_id, machine_id, start, end, duration))
                        print(f'  Thao tác {op_id} trên máy {machine_id}: bắt đầu {start}, kết thúc {end}, thời gian xử lý {duration}')
        return schedule_data
    else:
        print('Không tìm thấy lịch trình khả thi.')
        return None

# Chạy chương trình và lấy lịch trình
schedule = solve_fjss()

# Vẽ biểu đồ Gantt nếu có lịch trình
if schedule:
    import matplotlib.pyplot as plt

    # Tạo biểu đồ Gantt
    fig, ax = plt.subplots(figsize=(12, 6))

    # Màu sắc cho từng công việc
    colors = ['#ff9999', '#66b3ff', '#99ff99']  # Màu cho J0, J1, J2

    # Vẽ từng thao tác
    for job_id, op_id, machine_id, start, end, duration in schedule:
        ax.broken_barh([(start, end - start)], (machine_id - 0.4, 0.8), 
                       facecolors=colors[job_id], 
                       edgecolors='black', 
                       label=f'J{job_id}' if op_id == 0 else "")
        # Thêm nhãn cho thao tác
        ax.text(start + (end - start) / 2, machine_id, f'J{job_id}-O{op_id}', 
                ha='center', va='center',fontsize=20, color='black')

    # Thiết lập trục
    ax.set_ylim(-0.5, 2.5)  # 3 máy: M0, M1, M2
    ax.set_xlim(0, max(end for _, _, _, _, end, _ in schedule) + 1)
    ax.set_xlabel('Thời gian',fontsize=20, fontweight='bold')
    ax.set_yticks(range(3))
    ax.set_yticklabels([f'Machine {i}' for i in range(3)],fontsize=20, fontweight='bold')
    ax.grid(True, linestyle='--', alpha=0.7)

    # Thêm chú thích
    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys(), loc='upper right')

    # Thêm tiêu đề


    # Hiển thị biểu đồ
    plt.show()